In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.stats import zscore

# -----------------------------
# Utility Functions
# -----------------------------
def load_data(hdf5_file_path, session_id=None):
    """Load HDF5 file and attach a session_id column."""
    df = pd.read_hdf(hdf5_file_path, key='df')
    if session_id is not None:
        df['session_id'] = session_id
    return df

def normalize(v):
    norm = np.linalg.norm(v)
    return v if norm < 1e-8 else v / norm

def compute_euler_angles_from_row(row):
    """Compute head Euler angles (relative to body) for one frame."""
    def get_kp(idx):
        return np.array([row[f'kp{idx}_x'], row[f'kp{idx}_y'], row[f'kp{idx}_z']])
    
    # Head coordinate frame (EarL=1, EarR=2, Snout=3)
    earL = get_kp(1)
    earR = get_kp(2)
    snout = get_kp(3)
    ear_mid = (earL + earR) / 2.0
    head_x = normalize(snout - ear_mid)
    temp_y = earR - earL
    head_y = normalize(temp_y - np.dot(temp_y, head_x) * head_x)
    head_z = normalize(np.cross(head_x, head_y))
    
    # Body coordinate frame (SpineF=4, SpineM=5, ShoulderL=12, ShoulderR=16)
    spineF = get_kp(4)
    spineM = get_kp(5)
    shoulderL = get_kp(12)
    shoulderR = get_kp(16)
    body_x = normalize(spineF - spineM)
    shoulder_vec = shoulderR - shoulderL
    body_y = normalize(shoulder_vec - np.dot(shoulder_vec, body_x) * body_x)
    body_z = normalize(np.cross(body_x, body_y))
    
    # Compute relative rotation from body to head.
    R_body = np.column_stack((body_x, body_y, body_z))
    R_head = np.column_stack((head_x, head_y, head_z))
    R_rel = np.dot(R_body.T, R_head)
    euler_angles = R.from_matrix(R_rel).as_euler('xyz', degrees=True)
    return euler_angles

def compute_all_euler_angles(df):
    """Compute Euler angles for all frames and add them as columns to df."""
    nframes = len(df)
    yaw_vals   = np.empty(nframes)
    pitch_vals = np.empty(nframes)
    roll_vals  = np.empty(nframes)
    for i in range(nframes):
        row = df.iloc[i]
        euler = compute_euler_angles_from_row(row)
        yaw_vals[i]   = euler[0]
        pitch_vals[i] = euler[1]
        roll_vals[i]  = euler[2]
    # Unwrap angles and convert to degrees.
    yaw_vals = np.degrees(np.unwrap(np.radians(yaw_vals)))
    pitch_vals = np.degrees(np.unwrap(np.radians(pitch_vals)))
    roll_vals = np.degrees(np.unwrap(np.radians(roll_vals)))
    df['Yaw'] = yaw_vals
    df['Pitch'] = pitch_vals
    df['Roll'] = roll_vals
    return yaw_vals, pitch_vals, roll_vals

def get_neural_columns(df, prefix='dF_F_roi'):
    """Retrieve neural signal columns based on prefix."""
    return [col for col in df.columns if col.startswith(prefix)]

def exclude_neurons(df, neural_columns, exclude_indices, outlier_sigma=3):
    """
    Exclude neurons based on outlier criteria and manual indices.
    """
    # Outlier exclusion based on max activity.
    neuron_max_vals = {neuron: df[neuron].max() for neuron in neural_columns}
    max_array = np.array(list(neuron_max_vals.values()))
    mean_max = np.mean(max_array)
    std_max = np.std(max_array)
    threshold = mean_max + outlier_sigma * std_max
    outlier_neurons = [neuron for neuron, val in neuron_max_vals.items() if val > threshold]
    
    # Manual exclusion.
    manual_excluded_neurons = [f'dF_F_roi{i}' for i in exclude_indices]
    clean_neural_columns = [
        neuron for neuron in neural_columns 
        if (neuron not in outlier_neurons and neuron not in manual_excluded_neurons)
    ]
    return clean_neural_columns

def compute_tuning_curve(angle_vals, neural_vals, nbins=20):
    """
    Bin angle values and compute the mean neural response for each bin.
    """
    bins = np.linspace(np.min(angle_vals), np.max(angle_vals), nbins + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2.0
    avg_response = np.zeros(nbins)
    for i in range(nbins):
        mask = (angle_vals >= bins[i]) & (angle_vals < bins[i+1])
        avg_response[i] = np.mean(neural_vals[mask]) if np.any(mask) else np.nan
    return bin_centers, avg_response

def select_best_neuron(df, angle_vals, clean_neural_columns, nbins=20):
    """
    Select the neuron with the highest tuning curve variance for a given angle.
    """
    best_metric = -np.inf
    selected_neuron = None
    for neuron in clean_neural_columns:
        neural_vals = df[neuron].values
        _, tuning = compute_tuning_curve(angle_vals, neural_vals, nbins)
        metric = np.nanvar(tuning)
        if metric > best_metric:
            best_metric = metric
            selected_neuron = neuron
    return selected_neuron

def plot_tuning_curves(df, yaw_vals, pitch_vals, roll_vals, clean_neural_columns, best_neurons, nbins=20):
    """Plot tuning curves for Yaw, Pitch, and Roll."""
    fig, axes = plt.subplots(3, 1, figsize=(8, 12), sharex=False)
    angle_names = ['Yaw', 'Pitch', 'Roll']
    angle_vals_list = [yaw_vals, pitch_vals, roll_vals]
    
    for ax, angle_name, angle_vals in zip(axes, angle_names, angle_vals_list):
        # Plot all neurons in background.
        for neuron in clean_neural_columns:
            neural_vals = df[neuron].values
            centers, tuning = compute_tuning_curve(angle_vals, neural_vals, nbins)
            ax.plot(centers, tuning, linestyle='-', color='lightgray', alpha=0.5, zorder=1)
        
        # Highlight best neuron.
        neuron = best_neurons[angle_name]
        neural_vals = df[neuron].values
        centers, tuning = compute_tuning_curve(angle_vals, neural_vals, nbins)
        ax.plot(centers, tuning, linestyle='-', color='blue', lw=2.5, label=f'Best: {neuron}', zorder=2)
        ax.set_title(f'Tuning Curves for Head {angle_name}')
        ax.set_xlabel(f'{angle_name} (deg)')
        ax.set_ylabel('Neural Response (dF/F)')
        ax.legend()
        ax.grid(True)
    plt.tight_layout()
    plt.show()

def clustering_analysis(df, clean_neural_columns, num_clusters=2):
    """
    Cluster neurons based on their activity and plot cluster analysis views.
    """
    # Reset index without dropping so that 'timestamp_ms_mini' is preserved as a column.
    df_new = df.copy().reset_index()
    time = df_new['timestamp_ms_mini']
    neuron_activity = df_new[clean_neural_columns].values.T  # shape: (neurons, timepoints)
    
    # Drop low-variance neurons.
    neuron_variances = np.var(neuron_activity, axis=1)
    threshold = np.percentile(neuron_variances, 5)
    high_variance_indices = neuron_variances > threshold
    neuron_activity_filtered = neuron_activity[high_variance_indices, :]
    filtered_neural_columns = [col for i, col in enumerate(clean_neural_columns) if high_variance_indices[i]]
    
    # Z-score normalization.
    neuron_activity_normalized = zscore(neuron_activity_filtered, axis=1)
    Z = linkage(neuron_activity_normalized, method='average', metric='correlation')
    cluster_labels = fcluster(Z, t=num_clusters, criterion='maxclust')
    
    # Compute cluster mean activity.
    cluster_means = []
    for cluster_id in range(1, num_clusters + 1):
        cluster_neurons = neuron_activity_normalized[cluster_labels == cluster_id, :]
        cluster_mean = cluster_neurons.mean(axis=0)
        cluster_means.append(cluster_mean)
    cluster_means = np.array(cluster_means)
    
    # Plot cluster analysis results.
    fig = plt.figure(figsize=(18, 12))
    for cluster_id in range(num_clusters):
        row_offset = cluster_id * 3
        # 3D COM Trajectory.
        ax1 = fig.add_subplot(2, 3, row_offset + 1, projection='3d')
        sc1 = ax1.scatter(df_new['com_x'], df_new['com_y'], df_new['com_z'],
                          c=cluster_means[cluster_id, :len(df_new)], cmap='hot', marker='o')
        ax1.set_title(f'3D COM Trajectory (Cluster {cluster_id + 1})')
        ax1.set_xlabel('X Position')
        ax1.set_ylabel('Y Position')
        ax1.set_zlabel('Z Position')
        fig.colorbar(sc1, ax=ax1, pad=0.1, label='Mean Neural Activity (ΔF/F)')
        
        # XY Plane Projection.
        ax2 = fig.add_subplot(2, 3, row_offset + 2)
        sc2 = ax2.scatter(df_new['com_x'], df_new['com_y'],
                          c=cluster_means[cluster_id, :len(df_new)], cmap='hot', marker='o')
        ax2.set_title(f'XY Plane Projection (Cluster {cluster_id + 1})')
        ax2.set_xlabel('X Position')
        ax2.set_ylabel('Y Position')
        fig.colorbar(sc2, ax=ax2, label='Mean Neural Activity (ΔF/F)')
        
        # Z-Only View.
        ax3 = fig.add_subplot(2, 3, row_offset + 3)
        sc3 = ax3.scatter(time, df_new['com_z'],
                          c=cluster_means[cluster_id, :len(df_new)], cmap='hot', marker='o')
        ax3.set_title(f'Z-Only View (Cluster {cluster_id + 1})')
        ax3.set_xlabel('Time (ms)')
        ax3.set_ylabel('Z Position')
        fig.colorbar(sc3, ax=ax3, label='Mean Neural Activity (ΔF/F)')
    plt.tight_layout()
    plt.show()
    return cluster_labels, filtered_neural_columns

# -----------------------------
# Pipeline for a Single Session
# -----------------------------
def run_single_session(session_params):
    """
    Run the analysis pipeline for a single session.
    
    session_params should be a dictionary with keys:
      - 'file_path'
      - 'session_id'
      - 'exclude_indices': list of neuron indices to exclude manually
    """
    print(f"Processing session: {session_params.get('session_id', 'unknown')}")
    # Load data.
    df = load_data(session_params['file_path'], session_id=session_params.get('session_id'))
    
    # Compute Euler angles.
    yaw_vals, pitch_vals, roll_vals = compute_all_euler_angles(df)
    
    # Retrieve and clean neural columns.
    neural_columns = get_neural_columns(df)
    clean_neural_columns = exclude_neurons(df, neural_columns, session_params.get('exclude_indices', []))
    
    # Select the best neuron for each angle.
    best_neurons = {
        'Yaw': select_best_neuron(df, yaw_vals, clean_neural_columns),
        'Pitch': select_best_neuron(df, pitch_vals, clean_neural_columns),
        'Roll': select_best_neuron(df, roll_vals, clean_neural_columns)
    }
    
    # Plot tuning curves.
    plot_tuning_curves(df, yaw_vals, pitch_vals, roll_vals, clean_neural_columns, best_neurons)
    
    # Perform clustering analysis.
    cluster_labels, filtered_neural_columns = clustering_analysis(df, clean_neural_columns)
    
    return {
        'session_id': session_params.get('session_id'),
        'best_neurons': best_neurons,
        'cluster_labels': cluster_labels,
        'filtered_neural_columns': filtered_neural_columns
    }

# -----------------------------
# Main: Run Sessions with Exclusion from JSON
# -----------------------------
if __name__ == '__main__':
    # Base path to prepend to session relative paths.
    base_path = "/data/big_rim/rsync_dcc_sum/Oct3V1"
    
    # Load exclusion indices from JSON.
    exclude_file = "/home/lq53/mir_repos/BBOP/random_tests/25feb_more_corr_explo/neuro_exclude.json"
    with open(exclude_file, "r") as f:
        exclude_dict = json.load(f)
    
    # List of session relative paths.
    recordings = [
        '2024_10_25/20241002PMCr2_15_42',
        '2024_10_25/20241002PMCr2_17_05',
        '2024_10_14/20240916v1r1_16_37',
        '2024_10_14/20240916v1r1_16_53',
        '2024_10_14/20240916v1r2_14_30',
        '2024_10_14/20240916v1r2_15_58',
        '2024_11_01/20240910V1r_AO_12_50',
        '2024_11_06/20241015pmcr2_16_53'
    ]
    
    results = []
    for rec in recordings:
        # Construct the session directory and the full file path.
        session_dir = f"{base_path}/{rec}"
        file_path = f"{session_dir}/MIR_Aligned/aligned_predictions_with_ca_and_dF_F.h5"
        # Compute session id by replacing "/" with "_".
        session_id = rec.replace("/", "_")
        # Get exclude indices from JSON if present; otherwise, use an empty list.
        exclude_indices = exclude_dict.get(session_dir, [])
        
        session_params = {
            'file_path': file_path,
            'session_id': session_id,
            'exclude_indices': exclude_indices
        }
        print(f"\nRunning analysis for session: {session_id}")
        res = run_single_session(session_params)
        results.append(res)
    
    print("Session analysis completed.")


In [ ]:
#!/usr/bin/env python3
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.ndimage import binary_erosion
from scipy.sparse import csc_matrix
sys.path.append(os.path.abspath('../../'))
# Import your miniscope utilities.
from utlis.Ca_tools.roi_spike_vis_utlis import load_minian_data, calculate_dff, overlay_roi_edges_exclude

# -----------------------------
# Helper: Find Miniscope Path from Mapping
# -----------------------------
def find_minian_path_for_recording(recording, mapping_data):
    """
    Given a recording identifier (e.g. '2024_10_25/20241002PMCr2_15_42'),
    search mapping_data (from mini_to_rec_mapping.json) for a miniscope path
    whose key or its associated "rec_path" field contains the session ID.
    
    Returns the miniscope path if found, else None.
    """
    session_id = recording.split('/')[-1]
    for mini_path, mapping in mapping_data.items():
        rec_field = mapping.get("rec_path") or ""
        if session_id in mini_path or session_id in rec_field:
            return mini_path
    return None

# -----------------------------
# Helper: Check if neuron should be excluded
# -----------------------------
def is_excluded(neuron_name, exclusions):
    """
    Returns True if the neuron (e.g. 'dF_F_roi5') is in the exclusions list.
    The exclusions list may contain integers or strings (like 5, '5', or "dF_F_roi5").
    """
    try:
        roi_idx = int(neuron_name.replace("dF_F_roi", ""))
    except ValueError:
        return False
    return (roi_idx in exclusions) or (str(roi_idx) in exclusions) or (neuron_name in exclusions)

# -----------------------------
# Function: Cluster Rec Data & Overlay Miniscope ROIs with ROI labels
# -----------------------------
def overlay_clustered_roi_edges_from_rec_and_minian(rec_file_path, minian_path,
                                                    clean_neural_columns=None,
                                                    exclusions=None,
                                                    num_clusters=2):
    """
    Load rec data from rec_file_path to perform clustering on neural activity,
    _excluding_ neurons specified in 'exclusions', then load the miniscope data from
    minian_path and overlay ROI edges (red for cluster 1, blue for cluster 2) on the max projection.
    Additionally, the function labels each ROI at its centroid.
    
    Parameters:
      rec_file_path (str): Full path to the rec HDF5 file.
      minian_path (str): Directory path to the miniscope data.
      clean_neural_columns (list, optional): List of neural column names to use.
          If None, all columns starting with "dF_F_roi" will be used.
      exclusions (list, optional): List of ROI names or indices to exclude 
          (e.g., [5, 17] or ["dF_F_roi5", "dF_F_roi17"]). If None, no exclusions are applied.
      num_clusters (int, optional): Number of clusters to form (default is 2).
    
    Returns:
      roi_cluster_mapping (dict): Mapping from ROI name to cluster label (only for non-excluded neurons).
      rec_cluster_labels (np.array): Cluster labels from the rec data (for the filtered neurons).
    """
    from scipy.ndimage import binary_erosion  # Ensure binary_erosion is imported
    from scipy.stats import zscore
    from scipy.cluster.hierarchy import linkage, fcluster

    if exclusions is None:
        exclusions = []
    
    # --- 1. Load and Process Rec Data (with exclusions) ---
    df = pd.read_hdf(rec_file_path, key='df').reset_index()
    
    # Derive neural columns if not provided.
    if clean_neural_columns is None:
        neural_columns = [col for col in df.columns if col.startswith("dF_F_roi")]
    else:
        neural_columns = clean_neural_columns
    
    # Exclude neurons before clustering using the helper function.
    clean_neural_columns = [col for col in neural_columns if not is_excluded(col, exclusions)]
    if len(clean_neural_columns) == 0:
        raise ValueError("No neurons left after applying exclusions!")
    
    # Extract neural activity (transpose so shape becomes: neurons x timepoints).
    neuron_activity = df[clean_neural_columns].values.T
    
    # Drop low-variance neurons.
    neuron_variances = np.var(neuron_activity, axis=1)
    threshold = np.percentile(neuron_variances, 5)
    high_variance_mask = neuron_variances > threshold
    neuron_activity_filtered = neuron_activity[high_variance_mask, :]
    filtered_neural_columns = [col for i, col in enumerate(clean_neural_columns) if high_variance_mask[i]]
    
    # Z-score normalization.
    neuron_activity_normalized = zscore(neuron_activity_filtered, axis=1)
    
    # Hierarchical clustering.
    Z = linkage(neuron_activity_normalized, method='average', metric='correlation')
    rec_cluster_labels = fcluster(Z, t=num_clusters, criterion='maxclust')
    
    # Build mapping from ROI name to cluster label.
    roi_cluster_mapping = {roi: cl for roi, cl in zip(filtered_neural_columns, rec_cluster_labels)}
    
    # --- 2. Load Miniscope Data for the Overlay ---
    mini_timestamps = os.path.join(minian_path, 'timeStamps.csv')
    data, ts = load_minian_data(minian_path, mini_timestamps)
    
    try:
        max_proj = data['max_proj'].values
    except Exception as e:
        raise ValueError("Error retrieving max projection from miniscope data: " + str(e))
    
    # Ensure ROI footprints (data['A']) are in dense format.
    if hasattr(data['A'], 'toarray'):
        A_dense = data['A'].toarray()
    elif hasattr(data['A'], 'values'):
        A_dense = data['A'].values
    else:
        A_dense = data['A']
    
    # Create mapping from ROI name to index in miniscope data.
    # We assume the miniscope ROI names are "dF_F_roi0", "dF_F_roi1", ... in order.
    roi_mapping = {f"dF_F_roi{i}": i for i in range(A_dense.shape[0])}
    
    # --- 3. Build Cluster Overlays (Red for Cluster 1, Blue for Cluster 2) ---
    cluster1_overlay = np.zeros_like(max_proj, dtype=float)
    cluster2_overlay = np.zeros_like(max_proj, dtype=float)
    
    for roi_name, cl in roi_cluster_mapping.items():
        if roi_name not in roi_mapping:
            print(f"Warning: ROI {roi_name} not found in miniscope data, skipping.")
            continue
        i = roi_mapping[roi_name]
        # Construct the ROI mask.
        if A_dense.ndim == 3:
            roi_mask = A_dense[i] > 0
        else:
            roi_mask = A_dense[i].reshape(max_proj.shape) > 0
        # Compute ROI edge (mask XOR its binary erosion).
        roi_edge = roi_mask ^ binary_erosion(roi_mask)
        # Add the edge to the corresponding cluster overlay.
        if cl == 1:
            cluster1_overlay += roi_edge.astype(float)
        elif cl == 2:
            cluster2_overlay += roi_edge.astype(float)
    
    # Clip overlay values to [0, 1].
    cluster1_overlay = np.clip(cluster1_overlay, 0, 1)
    cluster2_overlay = np.clip(cluster2_overlay, 0, 1)
    
    # --- 4. Plot the Overlays with ROI Labels ---
    plt.figure(figsize=(10, 10))
    # Display the max projection without specifying a colormap to mimic your original look.
    plt.imshow(max_proj, interpolation='nearest')
    plt.imshow(cluster1_overlay, cmap='Reds', alpha=0.3, interpolation='nearest')
    plt.imshow(cluster2_overlay, cmap='Blues', alpha=0.3, interpolation='nearest')
    
    # Label each ROI at its centroid.
    for roi_name, cl in roi_cluster_mapping.items():
        if roi_name not in roi_mapping:
            continue
        idx = roi_mapping[roi_name]
        # Construct the ROI mask.
        if A_dense.ndim == 3:
            roi_mask = A_dense[idx] > 0
        else:
            roi_mask = A_dense[idx].reshape(max_proj.shape) > 0
        coords = np.argwhere(roi_mask)
        if coords.size > 0:
            centroid = coords.mean(axis=0)
            # Use red for cluster 1, blue for cluster 2.
            cluster_color = 'red' if cl == 1 else 'blue'
            # Extract the numeric part of the ROI name.
            roi_label = roi_name.replace("dF_F_roi", "")
            plt.text(centroid[1], centroid[0] + 20, roi_label, color=cluster_color,
                     fontsize=10, ha='center', va='center')
    
    plt.title('Max Projection with Clustered ROI Edges Overlay (Exclusions Applied)')
    plt.axis('off')
    plt.show()
    
    return roi_cluster_mapping, rec_cluster_labels


# -----------------------------
# Main Script: Example Usage
# -----------------------------
def main():
    # Define the base recor-+-ding path and list of recordings.
    base_path = "/data/big_rim/rsync_dcc_sum/Oct3V1"
    # recordings = [
    #     '2024_10_25/20241002PMCr2_15_42',
    #     '2024_10_25/20241002PMCr2_17_05',
    #     '2024_11_06/20241015pmcr2_16_53'
    # ]
    recordings = [
        '2024_10_25/20241002PMCr2_15_42',
        '2024_10_25/20241002PMCr2_17_05',
        '2024_10_14/20240916v1r1_16_37',
        '2024_10_14/20240916v1r1_16_53',
        '2024_10_14/20240916v1r2_14_30',
        '2024_10_14/20240916v1r2_15_58',
        '2024_11_01/20240910V1r_AO_12_50',
        '2024_11_06/20241015pmcr2_16_53'
    ]
    
    # Load the mapping JSON.
    mapping_file = "/home/lq53/mir_repos/BBOP/random_tests/25feb_more_corr_explo/mini_to_rec_mapping.json"
    with open(mapping_file, "r") as f:
        mapping_data = json.load(f)
    
    # Load the neuron exclusion JSON.
    exclude_file = "/home/lq53/mir_repos/BBOP/random_tests/25feb_more_corr_explo/neuro_exclude.json"
    with open(exclude_file, "r") as f:
        neuro_exclude = json.load(f)
    
    # Loop over each recording.
    for rec in recordings:
        rec_path = os.path.join(base_path, rec)
        print(f"\nProcessing recording: {rec_path}")
        
        # Find corresponding miniscope path.
        minian_path = find_minian_path_for_recording(rec, mapping_data)
        if minian_path is None:
            print(f"  Miniscope path not found for {rec}")
            continue
        print(f"  Miniscope path found: {minian_path}")
        
        # Construct the rec file path.
        rec_file_path = os.path.join(rec_path, "MIR_Aligned", "aligned_predictions_with_ca_and_dF_F.h5")
        
        # Get exclusions for this recording from the exclusion JSON.
        exclusions = neuro_exclude.get(rec_path, [])
        
        # --- Optionally: Overlay ROI edges with exclusions using miniscope data only ---
        try:
            mini_timestamps = os.path.join(minian_path, 'timeStamps.csv')
            data, ts = load_minian_data(minian_path, mini_timestamps)
            overlay_roi_edges_exclude(data, data['max_proj'].values, exclusions)
            print("  ROI edges overlaid with exclusions (miniscope only).")
        except Exception as e:
            print(f"  Error overlaying ROI edges with exclusions: {e}")
        
        # --- Perform rec data clustering & overlay clustered ROI edges with ROI labels ---
        try:
            roi_cluster_mapping, rec_cluster_labels = overlay_clustered_roi_edges_from_rec_and_minian(
                rec_file_path, minian_path, clean_neural_columns=None, exclusions=exclusions, num_clusters=2)
            print("  Auto-clustered ROI edges overlaid with ROI labels.")
        except Exception as e:
            print(f"  Error overlaying clustered ROI edges: {e}")
    
    print("Processing completed.")

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--aggregate]
ipykernel_launcher.py: error: unrecognized arguments: --f=/run/user/1001/jupyter/runtime/kernel-v3ce82c3530ce1049b220ef29f34e6aa2abd577d74.json


SystemExit: 2

/home/lq53/miniconda3/envs/bbop241209/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
